<h1>Previsão de empréstimo</h1>

Neste notebook contém um projeto desenvolvido durante o curso "Machine Learning e Data Science com Python de A a Z" da instituição Udemy.

OBJETIVO DA ANÁLISE:
- Utilizar algoritmos de Machine Learning para prever se o empréstimo será pago

Abaixo você pode conferir as colunas encontradas no arquivo “credit_data.csv”:

ATRIBUTOS PREVISORES:
- renda anual
- idade
- valor solicitado de empréstimo

ATRIBUTO CLASSE:
- pago = valor '0' para empréstimo não pago e 
  valor '1' para pago

In [1]:
import warnings
warnings.filterwarnings('ignore')

<h1>Importando a base de dados</h1>

In [2]:
import pandas as pd
import numpy as np

In [3]:
# importando a base de dados
base = pd.read_csv('credit_data.csv', names = ['cliente_id', 'renda_anual', 'idade', 'valor_emprestimo', \
                                               'pago'], header = 0)

In [4]:
base.head(10)

,cliente_id,renda_anual,idade,valor_emprestimo,pago
0,1,66155.925095,59.017015,8106.532131,0
1,2,34415.153966,48.117153,6564.745018,0
2,3,57317.170063,63.108049,8020.953296,0
3,4,42709.534201,45.751972,6103.642260,0
4,5,66952.688845,18.584336,8770.099235,1
5,6,24904.064140,57.471607,15.498598,0
6,7,48430.359613,26.809132,5722.581981,0
7,8,24500.141984,32.897548,2971.003310,1
8,9,40654.892537,55.496853,4755.825280,0
9,10,25075.872771,39.776378,1409.230371,0


<h1>Explorando os dados</h1>


In [5]:
# verificando o tipo de dados de cada coluna
base.dtypes

cliente_id            int64
renda_anual         float64
idade               float64
valor_emprestimo    float64
pago                  int64
dtype: object

In [6]:
# quantidade de linhas e colunas do dataFrame
base.shape
print("O dataset tem {} linhas e {} colunas.".format(base.shape[0], base.shape[1]))

O dataset tem 2000 linhas e 5 colunas.


In [7]:
# utilizando a função Info para obter um resumo sobre o dataset 
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   cliente_id        2000 non-null   int64  
 1   renda_anual       2000 non-null   float64
 2   idade             1997 non-null   float64
 3   valor_emprestimo  2000 non-null   float64
 4   pago              2000 non-null   int64  
dtypes: float64(3), int64(2)
memory usage: 78.2 KB


<h1>Tratamento de valores inconsistentes</h1> 

In [8]:
# Pode-se observar que existe na base de dados valores de idade negativos
# localizando os clientes com a idade negativa usando o método loc
base.loc[base['idade'] < 0]

,cliente_id,renda_anual,idade,valor_emprestimo,pago
15,16,50501.726689,-28.218361,3977.287432,0
21,22,32197.620701,-52.423280,4244.057136,0
26,27,63287.038908,-36.496976,9595.286289,0


In [10]:
# Para que os valores não fiquem incorretos, vamos substituir esses valores negativos pelo valor da média de idades
# calculando a média
base['idade'].mean()  # esse valor está incorreto pois foi considerado a média dos três valores inválidos

40.80755937840458

In [11]:
# calculando a média das idades excluindo os três valores incorretos
base['idade'][base.idade > 0].mean()

40.92770044906149

In [12]:
# substituindo os valores de idade incorretos pelo valor da média
base.loc[base.idade < 0, 'idade'] = 40.92

In [14]:
# verificando se os valores incorretos foram substituídos
base.loc[base['idade'] < 0]  # podemos observar que nenhum valor foi retornado

,cliente_id,renda_anual,idade,valor_emprestimo,pago


<h1>Tratamento de valores faltantes</h1>

In [15]:
# verificando se no campo idade existe valores nulos
pd.isnull(base['idade'])  # se retornar true significa que tem valores nulos

0       False
1       False
2       False
3       False
4       False
        ...  
1995    False
1996    False
1997    False
1998    False
1999    False
Name: idade, Length: 2000, dtype: bool

In [16]:
# com o comando anterior fica um pouco difícil vizualizar se existe valores nulos
# usando a função loc fica mais fácil identificar esses valores
# Podemos verificar que na base de dados existe três clientes que não estão com a idade preenchida.
base.loc[pd.isnull(base['idade'])]

,cliente_id,renda_anual,idade,valor_emprestimo,pago
28,29,59417.805406,NaN,2082.625938,0
30,31,48528.852796,NaN,6155.784670,0
31,32,23526.302555,NaN,2862.010139,0


In [17]:
# criando uma variável para armazenar os atributos previsores
# para a criação dessa variável não será incluído a coluna 'clientid', pois é um identificador e não é útil usar para a criação
# de algoritmos de machine leraning, pois não existe padrão
previsores = base.iloc[:, 1:4].values
previsores

array([[6.61559251e+04, 5.90170151e+01, 8.10653213e+03],
       [3.44151540e+04, 4.81171531e+01, 6.56474502e+03],
       [5.73171701e+04, 6.31080495e+01, 8.02095330e+03],
       ...,
       [4.43114493e+04, 2.80171669e+01, 5.52278669e+03],
       [4.37560566e+04, 6.39717958e+01, 1.62272260e+03],
       [6.94365796e+04, 5.61526170e+01, 7.37883360e+03]])

In [18]:
# criando uma variável para armazenar o atributo classe
classe = base.iloc[:, 4].values
classe

array([0, 0, 0, ..., 1, 0, 0], dtype=int64)

In [19]:
# tratando os valores nulos
# substituindo os valores nulos pelo valor da média
# a aplicação será feita pra todas as colunas de 'previsores' caso haja valores faltantes
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

# Substituindo os valores NaN pela média dos dados
previsores = imputer.fit_transform(previsores)

In [20]:
previsores

array([[6.61559251e+04, 5.90170151e+01, 8.10653213e+03],
       [3.44151540e+04, 4.81171531e+01, 6.56474502e+03],
       [5.73171701e+04, 6.31080495e+01, 8.02095330e+03],
       ...,
       [4.43114493e+04, 2.80171669e+01, 5.52278669e+03],
       [4.37560566e+04, 6.39717958e+01, 1.62272260e+03],
       [6.94365796e+04, 5.61526170e+01, 7.37883360e+03]])

<h1>Escalonamento dos atributos</h1>

In [21]:
# Aplicar uma fórmula para colocar a renda e a idade na mesma escala
# Utilizando a Padronização x= x - média(x) / desvioPadrão(x)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
previsores = scaler.fit_transform(previsores)

In [22]:
previsores

array([[ 1.45393393,  1.36538093,  1.20281942],
       [-0.76217555,  0.5426602 ,  0.69642695],
       [ 0.83682073,  1.67417189,  1.17471147],
       ...,
       [-0.07122592, -0.97448519,  0.35420081],
       [-0.11000289,  1.73936739, -0.92675625],
       [ 1.682986  ,  1.14917639,  0.96381038]])

<h1>Spliting</h1>

In [23]:
# Dividindo as variáveis em treino e teste
# 75% dados para treino e 25% dados para teste
from sklearn.model_selection import train_test_split
previsores_treinamento, previsores_teste, classe_treinamento, classe_teste = train_test_split(previsores, classe, \
                                                                                              test_size=0.25, random_state=0)

In [24]:
# Imprimindo os resultados
print("{0:0.2f}% nos dados de treino".format((len(previsores_treinamento)/len(base.index)) * 100))
print("{0:0.2f}% nos dados de teste".format((len(previsores_teste)/len(base.index)) * 100))

75.00% nos dados de treino
25.00% nos dados de teste


In [25]:
len(previsores_treinamento)

1500

In [26]:
len(classe_treinamento)

1500

In [27]:
len(previsores_teste)

500

In [28]:
len(classe_teste)

500

<h1>Naive Bayes</h1>

O algoritmo “Naive Bayes” é um classificador probabilístico.
Ele recebe o nome de “naive” (ingênuo) porque desconsidera a correlação entre as variáveis (features).

Classificação de textos, filtragem de SPAM e análise de sentimento em redes sociais são algumas das muitas aplicações para esse algoritmo.
O algoritmo é muito robusto para previsões em tempo real, pois precisa de poucos dados para realizar a classificação. Se existe a necessidade de correlacionar fatores, esse algoritmo tende a falhar nas predições.

### Construindo e treinando o modelo

In [29]:
# Importando o algoritmo
from sklearn.naive_bayes import GaussianNB

In [30]:
# Criando o modelo preditivo
classificador = GaussianNB()

In [31]:
# Treinando o modelo
classificador.fit(previsores_treinamento, classe_treinamento)

GaussianNB(priors=None, var_smoothing=1e-09)

In [32]:
# testando o modelo
previsoes = classificador.predict(previsores_teste)

### Verificando a exatidão no modelo

In [34]:
# precisão do modelo
from sklearn.metrics import confusion_matrix, accuracy_score
precisao = accuracy_score(classe_teste, previsoes)
precisao

0.938

In [36]:
# matriz de confusão
# diagonal principal (quantidade de acertos)
matriz = confusion_matrix(classe_teste, previsoes)
matriz

array([[428,   8],
       [ 23,  41]], dtype=int64)

<h1>Árvore de Decisão</h1>

Uma árvore de decisão utiliza base de dados histórica e geralmente começa com um único nó, que se divide em possíveis resultados. Cada um desses resultados leva a nós adicionais, que se ramificam em outras possibilidades. Assim, cria-se uma forma de árvore. Esses algoritmos são considerados um dos melhores e mais utilizados métodos de aprendizagem supervisionada, pois nos dão modelos preditivos de alta precisão, estabilidade e facilidade de interpretação. Ao contrário dos modelos lineares, eles mapeiam muito bem relações não-lineares.

### Construindo e treinando o modelo

In [37]:
# Importando o algoritmo
from sklearn.tree import DecisionTreeClassifier

In [38]:
# Criando o modelo preditivo
classificador = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)

In [39]:
# Treinando o modelo
classificador.fit(previsores_treinamento, classe_treinamento)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=0, splitter='best')

In [40]:
# testando o modelo
previsoes = classificador.predict(previsores_teste)

### Verificando a exatidão no modelo

In [42]:
# precisão do modelo
precisao = accuracy_score(classe_teste, previsoes)
precisao

0.982

In [43]:
# matriz de confusão
# diagonal principal (quantidade de acertos)
matriz = confusion_matrix(classe_teste, previsoes)
matriz

array([[430,   6],
       [  3,  61]], dtype=int64)

<h1>Random Forest</h1>

Esse algoritmo irá criar muitas árvores de decisão, de maneira aleatória, formando o que podemos enxergar como uma floresta, onde cada árvore será utilizada na escolha do resultado final. É um método de aprendizagem de máquina versátil e capaz de executar tarefas de regressão e de classificação. Ele também aplica métodos de redução dimensional, trata valores faltantes, valores anómalos (‘outliers’) e outras etapas essenciais da exploração de dados.
É um tipo de método de aprendizado de ‘ensemble’, onde um grupo de modelos fracos são combinados para formar um modelo mais forte.

### Construindo e treinando o modelo

In [44]:
# Importando o algoritmo
from sklearn.ensemble import RandomForestClassifier

In [45]:
# Criando o modelo preditivo
classificador = RandomForestClassifier(n_estimators = 40, criterion = 'entropy', random_state = 0)

In [46]:
# Treinando o modelo
classificador.fit(previsores_treinamento, classe_treinamento)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=40,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [47]:
# testando o modelo
previsoes = classificador.predict(previsores_teste)

### Verificando a exatidão no modelo

In [49]:
# precisão do modelo
precisao = accuracy_score(classe_teste, previsoes)
precisao

0.984

In [51]:
# matriz de confusão
# diagonal principal (quantidade de acertos)
matriz = confusion_matrix(classe_teste, previsoes)
matriz

array([[433,   3],
       [  5,  59]], dtype=int64)

<h1>KNN</h1>

O KNN (K-Nearest Neighbor) tem o objetivo de determinar a qual grupo uma determinada amostra vai pertencer com base nas amostras vizinhas. Os exemplos de treinamento são armazenados e a previsão é feita somente quando um novo registro precisa ser classificado. Diferente dos outros algoritmos, ele não constrói um modelo, apenas faz o cálculo da distância.

In [52]:
# Importando o algoritmo
from sklearn.neighbors import KNeighborsClassifier

In [53]:
classificador = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)

In [54]:
classificador.fit(previsores_treinamento, classe_treinamento)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [55]:
previsoes = classificador.predict(previsores_teste)

### Verificando a exatidão 

In [57]:
# precisão
precisao = accuracy_score(classe_teste, previsoes)
precisao

0.986

In [59]:
# matriz de confusão
# diagonal principal (quantidade de acertos)
matriz = confusion_matrix(classe_teste, previsoes)
matriz

array([[432,   4],
       [  3,  61]], dtype=int64)

<h1>Regressão Logística</h1>

Esse algoritmo mede a relação entre a variável dependente categórica e uma ou mais variáveis independentes, estimando as probabilidades usando uma função logística. Analisa diferentes aspectos ou variáveis de um objeto para depois determinar uma classe na qual ele se encaixa melhor.

### Construindo e treinando o modelo

In [60]:
# Importando o algoritmo
from sklearn.linear_model import LogisticRegression

In [61]:
# Criando o modelo preditivo
classificador = LogisticRegression(random_state = 1, solver='lbfgs')

In [62]:
# Treinando o modelo
classificador.fit(previsores_treinamento, classe_treinamento)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [63]:
# testando o modelo
previsoes = classificador.predict(previsores_teste)

### Verificando a exatidão no modelo

In [64]:
# precisão do modelo
precisao = accuracy_score(classe_teste, previsoes)
precisao

0.946

In [65]:
# matriz de confusão
# diagonal principal (quantidade de acertos)
matriz = confusion_matrix(classe_teste, previsoes)
matriz

array([[423,  13],
       [ 14,  50]], dtype=int64)

<h1>SVM - Máquinas de Vetores de Suporte</h1>

Uma máquina de vetores de suporte (SVM) desenvolve o modelo tomando as entradas de treinamento, mapeando elas no espaço multidimensional e utilizando regressão para encontrar um hiperplano (um hiperplano é uma superfície em espaço de n dimensões que o separa em duas metades de espaço) que melhor separa duas classes de entradas. Uma vez que esse modelo tenha sido treinada, ele é capaz de avaliar novas entradas em relação ao hiperplano divisor e classificá-las em uma entre duas categorias.

### Construindo e treinando o modelo

In [66]:
# Importando o algoritmo
from sklearn.svm import SVC

In [67]:
# Criando o modelo preditivo
classificador = SVC(kernel = 'rbf', random_state = 1, C = 2.0, gamma = 'auto')

In [68]:
# Treinando o modelo
classificador.fit(previsores_treinamento, classe_treinamento)

SVC(C=2.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=1, shrinking=True, tol=0.001,
    verbose=False)

In [69]:
# testando o modelo
previsoes = classificador.predict(previsores_teste)

### Verificando a exatidão no modelo

In [70]:
# precisão do modelo
precisao = accuracy_score(classe_teste, previsoes)
precisao

0.988

In [71]:
# matriz de confusão
# diagonal principal (quantidade de acertos)
matriz = confusion_matrix(classe_teste, previsoes)
matriz

array([[434,   2],
       [  4,  60]], dtype=int64)

<h1>Redes Neurais Artificiais</h1>

Redes Neurais Artificiais são técnicas computacionais que apresentam um modelo matemático inspirado na estrutura neural de organismos inteligentes e que adquirem conhecimento através da experiência. As redes neurais são compostas por várias unidades de processamento. Mesmo sendo usadas para resolverem problemas complexos, será utilizado como um exemplo.

## Redes Neurais com sklearn

### Construindo e treinando o modelo

In [72]:
# Importando o algoritmo
from sklearn.neural_network import MLPClassifier

In [73]:
# Criando o modelo preditivo
classificador = MLPClassifier(verbose = True, max_iter = 1000, tol = 0.000010, random_state = 0)

In [74]:
# Treinando o modelo
classificador.fit(previsores_treinamento, classe_treinamento)

Iteration 1, loss = 0.60888258
Iteration 2, loss = 0.54856950
Iteration 3, loss = 0.49705584
Iteration 4, loss = 0.45274374
Iteration 5, loss = 0.41460015
Iteration 6, loss = 0.38239381
Iteration 7, loss = 0.35450647
Iteration 8, loss = 0.33020542
Iteration 9, loss = 0.30880148
Iteration 10, loss = 0.29023265
Iteration 11, loss = 0.27346286
Iteration 12, loss = 0.25853354
Iteration 13, loss = 0.24513368
Iteration 14, loss = 0.23289527
Iteration 15, loss = 0.22197697
Iteration 16, loss = 0.21185939
Iteration 17, loss = 0.20279624
Iteration 18, loss = 0.19432492
Iteration 19, loss = 0.18666815
Iteration 20, loss = 0.17959682
Iteration 21, loss = 0.17304216
Iteration 22, loss = 0.16697104
Iteration 23, loss = 0.16135457
Iteration 24, loss = 0.15611580
Iteration 25, loss = 0.15122931
Iteration 26, loss = 0.14669229
Iteration 27, loss = 0.14264640
Iteration 28, loss = 0.13861131
Iteration 29, loss = 0.13496534
Iteration 30, loss = 0.13164507
Iteration 31, loss = 0.12845773
Iteration 32, los

Iteration 274, loss = 0.02101446
Iteration 275, loss = 0.02096842
Iteration 276, loss = 0.02088429
Iteration 277, loss = 0.02084772
Iteration 278, loss = 0.02067162
Iteration 279, loss = 0.02068262
Iteration 280, loss = 0.02054337
Iteration 281, loss = 0.02056139
Iteration 282, loss = 0.02037177
Iteration 283, loss = 0.02043931
Iteration 284, loss = 0.02026343
Iteration 285, loss = 0.02032413
Iteration 286, loss = 0.02015205
Iteration 287, loss = 0.02004596
Iteration 288, loss = 0.01998146
Iteration 289, loss = 0.01992813
Iteration 290, loss = 0.01990523
Iteration 291, loss = 0.01984882
Iteration 292, loss = 0.01973041
Iteration 293, loss = 0.01965528
Iteration 294, loss = 0.01961069
Iteration 295, loss = 0.01955367
Iteration 296, loss = 0.01946524
Iteration 297, loss = 0.01940217
Iteration 298, loss = 0.01936100
Iteration 299, loss = 0.01926885
Iteration 300, loss = 0.01921192
Iteration 301, loss = 0.01913168
Iteration 302, loss = 0.01906063
Iteration 303, loss = 0.01907268
Iteration 

Iteration 546, loss = 0.00993382
Iteration 547, loss = 0.00991252
Iteration 548, loss = 0.00992532
Iteration 549, loss = 0.00990903
Iteration 550, loss = 0.00992385
Iteration 551, loss = 0.00988578
Iteration 552, loss = 0.00978914
Iteration 553, loss = 0.00985931
Iteration 554, loss = 0.00984018
Iteration 555, loss = 0.00978697
Iteration 556, loss = 0.00976582
Iteration 557, loss = 0.00984114
Iteration 558, loss = 0.00969184
Iteration 559, loss = 0.00968946
Iteration 560, loss = 0.00966053
Iteration 561, loss = 0.00964940
Iteration 562, loss = 0.00965137
Iteration 563, loss = 0.00960377
Iteration 564, loss = 0.00956762
Iteration 565, loss = 0.00953344
Iteration 566, loss = 0.00958855
Iteration 567, loss = 0.00953720
Iteration 568, loss = 0.00946304
Iteration 569, loss = 0.00962997
Iteration 570, loss = 0.00942852
Iteration 571, loss = 0.00949459
Iteration 572, loss = 0.00954454
Iteration 573, loss = 0.00949557
Iteration 574, loss = 0.00942138
Iteration 575, loss = 0.00940756
Iteration 

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=1000,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=0, shuffle=True, solver='adam',
              tol=1e-05, validation_fraction=0.1, verbose=True,
              warm_start=False)

In [75]:
# testando o modelo
previsoes = classificador.predict(previsores_teste)

### Verificando a exatidão no modelo

In [76]:
# precisão do modelo
precisao = accuracy_score(classe_teste, previsoes)
precisao

0.996

In [77]:
# matriz de confusão
# diagonal principal (quantidade de acertos)
matriz = confusion_matrix(classe_teste, previsoes)
matriz

array([[435,   1],
       [  1,  63]], dtype=int64)

## Redes Neurais com Keras

### Construindo e treinando o modelo

In [106]:
from keras.models import Sequential
from keras.layers import Dense

In [107]:
# Criando o modelo preditivo
classificador = Sequential()

In [108]:
# Criação das camadas ocultas e de saída
classificador.add(Dense(units = 2, activation = 'relu', input_dim = 3))
classificador.add(Dense(units = 2, activation = 'relu'))
classificador.add(Dense(units = 1, activation = 'sigmoid'))

In [109]:
# Compilar a rede neural
classificador.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [110]:
# Treinando o modelo
classificador.fit(previsores_treinamento, classe_treinamento, batch_size = 10, epochs = 100)

Epoch 1/100
1500/1500 [==============================] - 0s 134us/step - loss: 0.6844 - accuracy: 0.8540
Epoch 2/100
1500/1500 [==============================] - 0s 62us/step - loss: 0.5861 - accuracy: 0.8540
Epoch 3/100
1500/1500 [==============================] - 0s 62us/step - loss: 0.5127 - accuracy: 0.8540
Epoch 4/100
1500/1500 [==============================] - 0s 63us/step - loss: 0.4490 - accuracy: 0.8540
Epoch 5/100
1500/1500 [==============================] - 0s 62us/step - loss: 0.3978 - accuracy: 0.8540
Epoch 6/100
1500/1500 [==============================] - 0s 62us/step - loss: 0.3562 - accuracy: 0.8540
Epoch 7/100
1500/1500 [==============================] - 0s 63us/step - loss: 0.3104 - accuracy: 0.8613
Epoch 8/100
1500/1500 [==============================] - 0s 62us/step - loss: 0.2630 - accuracy: 0.9013
Epoch 9/100
1500/1500 [==============================] - 0s 62us/step - loss: 0.2231 - accuracy: 0.9353
Epoch 10/100
1500/1500 [==============================] - 0s 64

1500/1500 [==============================] - 0s 65us/step - loss: 0.0207 - accuracy: 0.9967
Epoch 80/100
1500/1500 [==============================] - 0s 64us/step - loss: 0.0202 - accuracy: 0.9947
Epoch 81/100
1500/1500 [==============================] - 0s 64us/step - loss: 0.0199 - accuracy: 0.9967
Epoch 82/100
1500/1500 [==============================] - 0s 64us/step - loss: 0.0205 - accuracy: 0.9947
Epoch 83/100
1500/1500 [==============================] - 0s 65us/step - loss: 0.0196 - accuracy: 0.9953
Epoch 84/100
1500/1500 [==============================] - 0s 65us/step - loss: 0.0195 - accuracy: 0.9960
Epoch 85/100
1500/1500 [==============================] - 0s 64us/step - loss: 0.0190 - accuracy: 0.9967
Epoch 86/100
1500/1500 [==============================] - 0s 65us/step - loss: 0.0190 - accuracy: 0.9960
Epoch 87/100
1500/1500 [==============================] - 0s 65us/step - loss: 0.0189 - accuracy: 0.9960
Epoch 88/100
1500/1500 [==============================] - 0s 66us/st

In [111]:
# testando o modelo
previsoes = classificador.predict(previsores_teste)
previsoes = (previsoes > 0.5)

### Verificando a exatidão no modelo

In [112]:
# precisão do modelo
precisao = accuracy_score(classe_teste, previsoes)
precisao

1.0

In [113]:
# matriz de confusão
# diagonal principal (quantidade de acertos)
matriz = confusion_matrix(classe_teste, previsoes)
matriz

array([[436,   0],
       [  0,  64]], dtype=int64)

<h1>Conclusão</h1>

Realizando apenas um teste podemos observar que os Algoritmos de Redes Neurais Artificiais, SVM e Random Forest tiveram um maior percentual de acerto nessa base de dados. Isso não significa que esses algoritmos sejam os melhores a serem utilizados para prever se um empréstimo será pago. Para uma melhor compreensão desse resultado, deve ser realizado mais testes para comprovar o desempenho dos algoritmos.